# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import azureml
import joblib
import pickle
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig

from train import clean_data
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

experiment_name = 'Mushroom'
amlcompute_cluster_name = "comp"
ds_key = "Mushroom"
ds_description_text = "This dataset includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms"
project_folder = './mushroom-project'

## Workspace


In [ ]:
ws = Workspace.from_config()
experiment=Experiment(ws, experiment_name)

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')
run = experiment.start_logging()

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
found = False
if ds_key in ws.datasets.keys():
        found = True
        dataset = ws.datasets[ds_key]

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/mixmasteru/MLEND-capstone/main/data/mushrooms.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=ds_key,
                                   description=ds_description_text)
df = dataset.to_pandas_dataframe()
df.describe()
dataset.take(5).to_pandas_dataframe()



# Use the clean_data function to clean your data.
x, y = clean_data(df)
x["class"] = y
x_train, x_test = train_test_split(x, train_size=0.75, test_size=0.25, random_state=101)


Path("./input").mkdir(parents=True, exist_ok=True)
pd.DataFrame(x_train).to_csv("./input/x_train.csv", index=False)
pd.DataFrame(x_train).to_csv("./input/x_test.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./input', target_path='input', overwrite=True, show_progress=True)

x_train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('input/x_train.csv'))
x_test_data = TabularDatasetFactory.from_delimited_files(path=ds.path('input/x_test.csv'))


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "featurization": 'auto'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=x_train_data,
                             validation_data=x_test_data,
                             label_column_name="class",
                             path = project_folder,
                             enable_early_stopping= True,
                             model_explainability=True,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion(show_output=True)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve best model from Run
best_run, best_model = remote_run.get_output()
print(best_run)
print(best_run.get_metrics())
joblib.dump(best_model, 'outputs/automl_model.joblib')



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service